In [ ]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

In [1]:
# 查看个人持久化工作区文件
# !ls /home/mw/work/

PieChart.jpg			 model_42_2.hdf5
__pycache__			 model_42_2_finetune.hdf5
catboost_info			 model_42_3.hdf5
cbt_oof.csv			 model_42_3_finetune.hdf5
data_tmp			 models
deep_tools.py			 pytorch_pretrained
encoder.hdf5			 read_all_data.py
example.html			 submit.csv
feature.csv			 total_data.pkl
features.csv			 train.py
kaggle_util			 train_sample
lgb_oof.csv			 utils.py
lgb_oof_feature_engineering.csv  wiki.zh.text
lost+found			 xgb_oof
model_42_0.hdf5			 xgb_oof.csv
model_42_0_finetune.hdf5	 基金近三月.xlsx
model_42_1.hdf5			 第一名基金近30个交易日净值情况.xlsx
model_42_1_finetune.hdf5	 逾期分布.png


In [4]:
# 查看当前kernel下的package
# !pip list --format=columns

time: 313 µs


In [5]:
# 显示cell运行时长
%load_ext klab-autotime

The klab-autotime extension is already loaded. To reload it, use:
  %reload_ext klab-autotime
time: 902 µs


In [6]:
import pandas as pd
import gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import lightgbm as lgb
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold,StratifiedKFold

time: 994 ms


In [7]:
df_train = pd.read_csv('/home/mw/input/tianchi6504/model_sample.csv')
df_train.reset_index()

df_train.head()

,user_id,y,x_001,x_002,x_003,x_004,x_005,x_006,x_007,x_008,...,x_190,x_191,x_192,x_193,x_194,x_195,x_196,x_197,x_198,x_199
0,A00002,0,0.0,32.0,0,0,0,0,0,0,...,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,A00005,0,0.0,29.0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A00006,0,0.0,31.0,0,0,0,0,0,0,...,2.0,2.0,4.0,4.0,6.0,6.0,6.0,6.0,10.0,10.0
3,A00008,0,0.0,22.0,0,0,0,0,0,0,...,3.0,3.0,3.0,3.0,5.0,5.0,3.0,3.0,7.0,7.0
4,A00009,0,0.0,31.0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


time: 233 ms


In [8]:
no_features = ['y','user_id']
features = [col for col in df_train.columns if col not in no_features]
ycol = 'y'

time: 725 µs


In [9]:
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.feature_selection import SelectFromModel

# clf = ExtraTreesClassifier(n_estimators=50)
# clf = clf.fit(train, target)

# model = SelectFromModel(clf, prefit=True)
# train_sel = model.transform(train)
# test_sel = model.transform(test)
# print('训练数据未特征筛选维度', train.shape)
# print('训练数据特征筛选维度后', train_sel.shape)


time: 316 µs


In [10]:
def get_features_middle(data):
    model_sample_strong_feature = data.copy()
    # 将身份信息以及财产信息进行编码
    first_strong_features = ['x_003','x_004','x_005','x_006','x_007','x_008','x_009','x_010','x_011','x_012','x_013','x_014','x_015','x_016','x_017','x_018','x_019']
    res = 0
    for i in range(len(first_strong_features)):
        res += 2 ** i * data[first_strong_features[i]]

    model_sample_strong_feature['x_1_strong'] = res 
    # 借记卡的比例特征
    model_sample_strong_feature['x_022/x_020'] = data['x_022'] / (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_023/x_020'] = data['x_023'] / (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_024/x_020'] = data['x_024'] /  (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_025/x_020'] = data['x_025'] /  (data['x_020']+ 1e-10)
    model_sample_strong_feature['x_026/x_020'] = data['x_026'] /  (data['x_020'] + 1e-10)
    
    # 贷记卡的比例特征
    model_sample_strong_feature['x_028/x_021'] = data['x_028'] / (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_029/x_021'] = data['x_029'] / (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_030/x_021'] = data['x_030'] /  (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_031/x_021'] = data['x_031'] /  (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_032/x_021'] = data['x_032'] /  (data['x_021'] + 1e-10)
    
    # 银行卡的比例特征
    model_sample_strong_feature['all_cards'] = (data['x_034'] +  data['x_035'] + data['x_036'] + data['x_037'] + data['x_038'] + data['x_039'] + data['x_040']  ).values

    model_sample_strong_feature['x_034/all_cards'] = data['x_034'] / (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_035/all_cards'] = data['x_035'] / (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_036/all_cards'] = data['x_036'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_037/all_cards'] = data['x_037'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_038/all_cards'] = data['x_038'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_039/all_cards'] = data['x_039'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_040/all_cards'] = data['x_040'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
   
    # 标准差还原
    model_sample_strong_feature['x_043/x_044'] = data['x_043'] / (data['x_044'] + 1e-10)
    model_sample_strong_feature['x_046/x_047'] = data['x_046'] / (data['x_047'] + 1e-10)
    model_sample_strong_feature['x_050/x_051'] = data['x_050'] / (data['x_051'] + 1e-10)
    model_sample_strong_feature['x_053/x_054'] = data['x_053'] / (data['x_054'] + 1e-10)
    model_sample_strong_feature['x_057/x_058'] = data['x_057'] / (data['x_058'] + 1e-10)
    model_sample_strong_feature['x_060/x_061'] = data['x_060'] / (data['x_061'] + 1e-10)
    model_sample_strong_feature['x_076/x_077'] = data['x_076'] / (data['x_077'] + 1e-10)
    model_sample_strong_feature['x_079/x_080'] = data['x_079'] / (data['x_080'] + 1e-10)
    model_sample_strong_feature['x_083/x_084'] = data['x_083'] / (data['x_084'] + 1e-10)
    model_sample_strong_feature['x_086/x_087'] = data['x_086'] / (data['x_087'] + 1e-10)
    model_sample_strong_feature['x_090/x_091'] = data['x_090'] / (data['x_091'] + 1e-10)
    model_sample_strong_feature['x_094/x_095'] = data['x_094'] / (data['x_095'] + 1e-10)
    model_sample_strong_feature['x_098/x_099'] = data['x_098'] / (data['x_099'] + 1e-10)
    model_sample_strong_feature['x_123/x_124'] = data['x_123'] / (data['x_124'] + 1e-10)
    model_sample_strong_feature['x_126/x_127'] = data['x_126'] / (data['x_127'] + 1e-10)

    
    # 每张卡（信用or其他）交易金额等；每笔（异地每笔）交易金额等；每笔还款金额等；每笔商旅，保险，家装，金融等的均值特征；每个月的平均交易笔数；其他有意义的均值特征
    
    model_sample_strong_feature['x_045/x_41'] = data['x_045'] / (data['x_041'] + 1e-10)
    model_sample_strong_feature['x_052/x_48'] = data['x_052'] / (data['x_048'] + 1e-10)
    model_sample_strong_feature['x_059/x_55'] = data['x_059'] / (data['x_055'] + 1e-10) 
    model_sample_strong_feature['x_064/x_062'] = data['x_064'] / (data['x_062'] + 1e-10)
    model_sample_strong_feature['x_067/x_065'] = data['x_067'] / (data['x_065'] + 1e-10)  
    model_sample_strong_feature['x_070/x_068'] = data['x_070'] / (data['x_068'] + 1e-10)
    model_sample_strong_feature['x_073/x_071'] = data['x_073'] / (data['x_071'] + 1e-10) 
    model_sample_strong_feature['x_078/x_074'] = data['x_078'] / (data['x_074'] + 1e-10)  
    model_sample_strong_feature['x_085/x_081'] = data['x_085'] / (data['x_081'] + 1e-10) 
    model_sample_strong_feature['x_100/x_101'] = data['x_100'] / (data['x_101'] + 1e-10)
    model_sample_strong_feature['x_102/x_103'] = data['x_102'] / (data['x_103'] + 1e-10) 
    model_sample_strong_feature['x_108/x_105'] = data['x_108'] / (data['x_105'] + 1e-10)
    model_sample_strong_feature['x_104/x_102'] = data['x_104'] / (data['x_102'] + 1e-10) 
    model_sample_strong_feature['x_109/x_110'] = data['x_109'] / (data['x_110'] + 1e-10)
    model_sample_strong_feature['x_111/x_109'] = data['x_111'] / (data['x_109'] + 1e-10) 
    model_sample_strong_feature['x_112/x_113'] = data['x_112'] / (data['x_113'] + 1e-10)
    model_sample_strong_feature['x_114/x_112'] = data['x_114'] / (data['x_112'] + 1e-10) 
    model_sample_strong_feature['x_115/x_116'] = data['x_115'] / (data['x_116'] + 1e-10)
    model_sample_strong_feature['x_117/x_115'] = data['x_117'] / (data['x_115'] + 1e-10)  
    model_sample_strong_feature['x_118/x_119'] = data['x_118'] / (data['x_119'] + 1e-10)
    model_sample_strong_feature['x_120/x_118'] = data['x_120'] / (data['x_118'] + 1e-10) 
    model_sample_strong_feature['x_125/x_121'] = data['x_125'] / (data['x_121'] + 1e-10) 
    model_sample_strong_feature['x_128/x_129'] = data['x_128'] / (data['x_129'] + 1e-10)
    model_sample_strong_feature['x_130/x_128'] = data['x_130'] / (data['x_128'] + 1e-10)

    # 每笔放款金额，每个机构的放款笔数，每个机构的放款金额
    model_sample_strong_feature['x_133/x_134'] = data['x_133'] / (data['x_134'] + 1e-10)
    model_sample_strong_feature['x_133/x_132'] = data['x_133'] / (data['x_132'] + 1e-10)
    model_sample_strong_feature['x_134/x_132'] = data['x_134'] / (data['x_132'] + 1e-10) 
    model_sample_strong_feature['x_138/x_139'] = data['x_138'] / (data['x_139'] + 1e-10)
    model_sample_strong_feature['x_138/x_137'] = data['x_138'] / (data['x_137'] + 1e-10)
    model_sample_strong_feature['x_139/x_137'] = data['x_139'] / (data['x_137'] + 1e-10) 
    model_sample_strong_feature['x_143/x_142'] = data['x_143'] / (data['x_142'] + 1e-10)
    model_sample_strong_feature['x_143/x_144'] = data['x_143'] / (data['x_144'] + 1e-10)
    model_sample_strong_feature['x_144/x_142'] = data['x_144'] / (data['x_142'] + 1e-10)

    # 每个机构的放款均值,失败还款笔数占比
    model_sample_strong_feature['x_151/x_149'] = data['x_151'] / (data['x_149'] + 1e-10)
    model_sample_strong_feature['x_152/x_149'] = data['x_152'] / (data['x_149'] + 1e-10)
    model_sample_strong_feature['x_152/x_151'] = data['x_152'] / (data['x_151'] + 1e-10)
    model_sample_strong_feature['x_154/x_153'] = data['x_154'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_156/x_153'] = data['x_156'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_157/x_153'] = data['x_157'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_158/x_153'] = data['x_158'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_159/x_153'] = data['x_159'] / (data['x_153'] + 1e-10)  
    model_sample_strong_feature['x_154/x_155'] = data['x_154'] / (data['x_155'] + 1e-10)  

    model_sample_strong_feature['x_164/x_162'] = data['x_164'] / (data['x_162'] + 1e-10)
    model_sample_strong_feature['x_165/x_162'] = data['x_165'] / (data['x_162'] + 1e-10)
    model_sample_strong_feature['x_165/x_164'] = data['x_165'] / (data['x_164'] + 1e-10)
    model_sample_strong_feature['x_167/x_166'] = data['x_167'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_169/x_166'] = data['x_169'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_170/x_166'] = data['x_170'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_171/x_166'] = data['x_171'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_180/x_181'] = data['x_180'] / (data['x_181'] + 1e-10) 
    model_sample_strong_feature['x_167/x_168'] = data['x_167'] / (data['x_168'] + 1e-10) 
    model_sample_strong_feature['x_172/x_167'] = data['x_172'] / (data['x_167'] + 1e-10)  

    model_sample_strong_feature['x_177/x_175'] = data['x_177'] / (data['x_175'] + 1e-10)
    model_sample_strong_feature['x_178/x_175'] = data['x_178'] / (data['x_175'] + 1e-10)
    model_sample_strong_feature['x_178/x_177'] = data['x_178'] / (data['x_177'] + 1e-10)
    model_sample_strong_feature['x_180/x_179'] = data['x_180'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_182/x_179'] = data['x_182'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_183/x_179'] = data['x_183'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_184/x_179'] = data['x_184'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_180/x_181'] = data['x_180'] / (data['x_181'] + 1e-10) 
    model_sample_strong_feature['x_185/x_180'] = data['x_185'] / (data['x_180'] + 1e-10)
 
    # 90天与30天的申请贷款机构的趋势，180天与90天的申请贷款机构的趋势，180天与30天的申请贷款机构的趋势；90天与30天的成功申请贷款机构的趋势，180天与90天的成功申请贷款机构的趋势，180天；
    # 30天的成功申请贷款机构的趋势；90天与30天的申请贷款笔数的趋势，180天与90天的申请贷款笔数的趋势，180天与30天的申请贷款笔数的趋势90天的申请贷款笔数的趋势
    model_sample_strong_feature['x_189/x_188'] = data['x_189'] / (data['x_188'] + 1e-10)
    model_sample_strong_feature['x_191/x_190'] = data['x_191'] / (data['x_190'] + 1e-10)
    model_sample_strong_feature['x_193/x_192'] = data['x_193'] / (data['x_192'] + 1e-10)
    model_sample_strong_feature['x_195/x_194'] = data['x_195'] / (data['x_194'] + 1e-10)
    model_sample_strong_feature['x_197/x_196'] = data['x_197'] / (data['x_196'] + 1e-10)
    model_sample_strong_feature['x_199/x_198'] = data['x_199'] / (data['x_198'] + 1e-10)
    model_sample_strong_feature['x_196/x_188'] = data['x_196'] / (data['x_188'] + 1e-10)
    model_sample_strong_feature['x_192/x_188'] = data['x_192'] / (data['x_188'] + 1e-10)
                                                                   
    model_sample_strong_feature = model_sample_strong_feature.fillna(-999)
    return model_sample_strong_feature 

time: 53.6 ms


In [11]:
features = get_features_middle(df_train)

time: 223 ms


In [12]:
features.y.head()

0    0
1    0
2    0
3    0
4    0
Name: y, dtype: int64

time: 2.46 ms


In [13]:
features.shape

(11017, 303)

time: 1.39 ms


In [14]:
features.to_csv('features.csv')

time: 2.95 s
